In [1]:
import copy

import jax
import clrs
import numpy as np

%load_ext autoreload
%autoreload 2

rng = np.random.RandomState(1234)
rng_key = jax.random.PRNGKey(rng.randint(2 ** 32))


2023-06-12 10:07:03.756387: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [1]:
# If you don't want BipartiteMatching, just pass empty generator list and
# length separately

train_sampler_spec = {
    'num_samples': 100,
    'batch_size': 32,
    'schematics': [
        {
            'generator': 'ER',
            'proportion': 1,
            'length': 16,
            'kwargs': {'low': 0, 'high': 1, 'weighted': True}
        },
        {
            'generator': 'ER',
            'proportion': 0,
            'length': 10,
            'length_2': 50,
            'kwargs': {'p': 0.1, 'low': 0, 'high': 1, 'weighted': True}
        },
    ]
}

test_sampler_spec = {
    'num_samples': 40,
    'batch_size': 40,
    'schematics': [
        {
            'generator': 'ER',
            'proportion': 0,
            'length': 100,
            'kwargs': {'p': 0.05, 'low': 0, 'high': 1, 'weighted': True}
        },
        {
            'generator': 'ER',
            'proportion': 1,
            'length': 64,
            'kwargs': {'low': 0, 'high': 1, 'weighted': True}
        },
        {
            'generator': 'ER',
            'proportion': 0,
            'length': 20,
            'length_2': 200,
            'kwargs': {'p': 0.2, 'low': 0, 'high': 1, 'weighted': True}
        },
    ]
}

def samplers(sampler_spec, **kwargs):
    batch_size = sampler_spec.get('batch_size', 1)
    num_samples = sampler_spec['num_samples']
    if batch_size > num_samples:
        batch_size = num_samples

    def _iterate_sampler(sampler, batch_size):
        while True:
            yield sampler.next(batch_size)

    sampler, spec = clrs.build_sampler(
        name = 'simplified_min_sum',
        sampler_spec = sampler_spec,
        **kwargs)  # number of nodes

    sampler = _iterate_sampler(sampler, batch_size = batch_size)
    return sampler, spec

train_sampler, spec = samplers(train_sampler_spec)
test_sampler, _ = samplers(test_sampler_spec)

NameError: name 'clrs' is not defined

In [5]:
sample = next(test_sampler)
sample.features.inputs[1].data[0].shape

(64, 64)

In [21]:
def define_model(spec, train_sampler, model = "mpnn"):
    if model == "mpnn":
        processor_factory = clrs.get_processor_factory('mpnn', use_ln = True,
                                                   nb_triplet_fts = 4)  #use_ln => use layer norm
    elif model == "gat":
        processor_factory = clrs.get_processor_factory('gat', use_ln=True, nb_heads = 4, nb_triplet_fts = 4)

    elif model == "mpnndoublemax":
        processor_factory = clrs.get_processor_factory('mpnndoublemax', use_ln = True, nb_triplet_fts = 0)  #use_ln => use layer norm

    elif model == "gmpnn":
        processor_factory = clrs.get_processor_factory('gmpnn', use_ln = True, nb_triplet_fts = 4)  #use_ln => use layer norm
    elif model == "pgn":
        processor_factory = clrs.get_processor_factory('pgn', use_ln = True, nb_triplet_fts = 32)  #use_ln => use layer norm
    elif model == "triplet_pgn_mask":
        processor_factory = clrs.get_processor_factory('triplet_pgn_mask', use_ln = True, nb_triplet_fts = 32)  #use_ln => use layer norm

    model_params = dict(
        processor_factory = processor_factory,  # contains the processor_factory
        hidden_dim = 32,  # TODO put back to 32 if no difference
        encode_hints = True,
        decode_hints = True,
        #decode_diffs=False,
        #hint_teacher_forcing_noise=1.0,
        hint_teacher_forcing = 1.0,
        use_lstm = False,
        learning_rate = 0.001,
        checkpoint_path = '/tmp/checkpt',
        freeze_processor = False,  # Good for post step
        dropout_prob = 0.5,
        # nb_msg_passing_steps=3,
    )

    dummy_trajectory = next(train_sampler)  # jax needs a trajectory that is plausible looking to init

    model = clrs.models.BaselineModel(
        spec = spec,
        dummy_trajectory = dummy_trajectory,
        **model_params
    )

    model.init(dummy_trajectory.features, 1234)  # 1234 is a random seed

    return model

model = define_model(spec, train_sampler, "triplet_pgn_mask")

In [9]:
# No evaluation since we are postprocessing with soft: TO CHANGE -> baselines.py line 336 outs change hard to False
# step = 0
#
# while step <= 1:
#     feedback, test_feedback = next(train_sampler), next(test_sampler)
#     rng_key, new_rng_key = jax.random.split(rng_key) # jax needs new random seed at step
#     cur_loss = model.feedback(rng_key, feedback) # loss is contained in model somewhere
#     rng_key = new_rng_key
#     if step % 10 == 0:
#         print(step)
#     step += 1



In [10]:
def train(model, epochs, train_sampler, test_sampler):
    step = 0
    rng_key = jax.random.PRNGKey(rng.randint(2 ** 32))

    while step <= epochs:
        feedback, test_feedback = next(train_sampler), next(test_sampler)
        # TODO remove - testing if uses hints on tests
        # shape = test_feedback.features.hints[0].data[0].shape
        # test_feedback.features.hints[0].data = test_feedback.features.hints[0].data[0, :, :].reshape((1, *shape))

        rng_key, new_rng_key = jax.random.split(rng_key)  # jax needs new random seed at step
        cur_loss = model.feedback(rng_key, feedback)  # loss is contained in model somewhere
        rng_key = new_rng_key
        if step % 50 == 0:
            predictions_val, _ = model.predict(rng_key, feedback.features)
            out_val = clrs.evaluate(feedback.outputs, predictions_val)
            predictions, _ = model.predict(rng_key, test_feedback.features)
            out = clrs.evaluate(test_feedback.outputs, predictions)
            print(
                f'step = {step} | loss = {cur_loss} | val_acc = {out_val["score"]} | test_acc = {out["score"]}')  # here, val accuracy is actually training accuracy, not great but is example
        if step % 150 == 0:
            learned, greedy = matching_value(test_feedback, predictions, partial = False, match_rest = False, opt_scipy = True)
            print(f"**learned: {learned}, greedy: {greedy}**")
        step += 1
    return model

In [11]:
from scipy.optimize import linear_sum_assignment


def matching_value(samples, predictions, partial = False, match_rest = False, opt_scipy = False):
    features = samples.features
    gt_matchings = samples.outputs[0].data
    # inputs for the matrix A are at index 1 (see spec.py)
    data = features.inputs[1].data
    masks = features.inputs[3].data
    pred_accuracy = 0
    greedy_accuracy = 0

    #TODO remove
    def _add_uniform_weights(adj, low, high):
        n, m = adj.shape
        weights = np.random.uniform(
            low=low, high=high, size=(n, m)
        )
        return adj * high + low

    # Iterating over all the samples
    for i in range(data.shape[0]):

        if opt_scipy:
            row_ind, col_ind = linear_sum_assignment(data[i], maximize = True)
            max_weight = data[i][row_ind, col_ind].sum() / 2  #TODO why /2
        else:
            max_weight = compute_greedy_matching_weight(i, data, masks, gt_matchings[i])

        predicted_matching = predictions["match"].data[i]

        if partial:
            preds_weight = compute_partial_matching_weight(i, data, masks, predicted_matching)
            # print(f"opt: {max_weight}, greedy learned: {preds_weight}")
        else:
            preds_weight = compute_greedy_matching_weight(i, data, masks, predicted_matching, match_rest = match_rest)
            # print(f"opt: {max_weight}, partial: {preds_weight}")

        # assert preds_weight <= max_weight
        greedy_matching_weight = naive_greedy(i, data, masks)
        # print(f"Naive greedy: {greedy_matching_weight}")
        greedy_accuracy += greedy_matching_weight / max_weight
        pred_accuracy += preds_weight / max_weight

    return pred_accuracy / data.shape[0], greedy_accuracy / data.shape[0]

def naive_greedy(i, data, masks):
    """Computes a matching greedily by, for each node, adding the maximum neighbor that
    hasn't yet been added to the matching"""

    matching_weight = 0
    A = data[i]
    buyers_mask = masks[i]
    n = int(np.sum(buyers_mask))
    # At the start, all the right hand side values are possible matches
    matching_mask = np.full(A.shape[0], True)

    # for buyer in range(n):
    #     # Checking if there are more elements to match (if more buyers than goods)
    #     if A[buyer, matching_mask].shape[0] != 0:
    #         matching_weight += np.max(A[buyer, matching_mask])
    #         # Recovering the index of the maximum, inspired by http://seanlaw.github.io/2015/09/10/numpy-argmin-with-a-condition/
    #         subset_idx = np.argmax(A[buyer, matching_mask])
    #         good = np.arange(A.shape[1])[matching_mask][subset_idx]
    #         # The corresponding good cannot be used anymore
    #         matching_mask[good] = False

    # Second method of computing a greedy matching
    # Set of vertices already matched
    matching = set()
    # Get the indices of the weights in highest to lowest order (hence the negative sign), inspired by https://stackoverflow.com/questions/30577375/have-numpy-argsort-return-an-array-of-2d-indices
    indices = np.dstack(np.unravel_index(np.argsort(- A.ravel()), A.shape))
    for index in indices[0]:
        if index[0] not in matching and index[1] not in matching:
            matching_weight += A[tuple(index)]
            matching.add(index[0])
            matching.add(index[1])


    return matching_weight




def compute_greedy_matching_weight(i, data, masks, matching, match_rest = False):
    matching_weight = 0
    A = data[i]
    buyers_mask = masks[i]
    n = int(np.sum(buyers_mask))
    goods_mask = 1 - buyers_mask
    m = int(np.sum(goods_mask))

    # Only consider the matching values for consumers
    matching = np.where(goods_mask == 1, matching, -1)
    unmatched_goods = set(range(n, n + m))
    unmatched_buyers = set(range(n))

    for buyer in range(n):
        if buyer in matching:
            # If several goods point to the same buyer, keep the one with maximum weight
            mask = matching == buyer
            matching_weight += np.max(A[buyer, mask])
            # Recovering the index of the maximum, inspired by http://seanlaw.github.io/2015/09/10/numpy-argmin-with-a-condition/
            subset_idx = np.argmax(A[buyer, mask])
            good = np.arange(A.shape[1])[mask][subset_idx]
            unmatched_goods.remove(good)
            unmatched_buyers.remove(buyer)

    if match_rest and len(unmatched_goods) > 0 and len(unmatched_buyers) > 0:
        # Compute optimal matching on the remaining unmatched nodes
        mask = np.zeros(A.shape)
        # TODO this is a horrible solution, there's definitely a prettier solution
        mask[list(unmatched_buyers)] += 1
        mask[:, list(unmatched_goods)] += 1
        mask = np.where(mask == 2, True, False)
        remaining_bipartite_graph = A * mask
        row_ind, col_ind = linear_sum_assignment(remaining_bipartite_graph, maximize = True)
        opt = A[row_ind, col_ind].sum() / 2  #TODO do I always need the division by 2
        matching_weight += opt

    return matching_weight


def compute_partial_matching_weight(i, data, masks, matching):
    # Matching is expected to be a (n+m)x(n+m) matrix where each row sums to 1 (weights assigned to other nodes)

    matching_weight = 0
    A = data[i]
    buyers_mask = masks[i]
    n = int(np.sum(buyers_mask))
    goods_mask = 1 - buyers_mask
    m = int(np.sum(goods_mask))

    # We only care about the buyer -> good connections
    A_submatrix = A[:n, n:n + m]
    matching = matching[:n, n:n + m]

    max_weight = np.max(np.sum(matching, axis = 0))
    # print(f"max weight: {max_weight}")
    matching /= max_weight
    return np.sum(matching * A_submatrix)

In [ ]:
model = train(model, 400, train_sampler, test_sampler)

step = 0 | loss = 5.347200870513916 | val_acc = 0.310546875 | test_acc = 0.30585938692092896
**learned: 0.3276043090977613, greedy: 0.9540878531495511**
step = 50 | loss = 0.678571879863739 | val_acc = 0.560546875 | test_acc = 0.536328136920929
step = 100 | loss = 0.4104996919631958 | val_acc = 0.572265625 | test_acc = 0.4996093809604645
step = 150 | loss = 0.2140585482120514 | val_acc = 0.4375 | test_acc = 0.2621093690395355
**learned: 0.8161358912510988, greedy: 0.9559042388299057**


In [19]:
test_feedback = next(test_sampler)


In [20]:
# test_feedback = next(test_sampler)
predictions, _ = model.predict(rng_key, test_feedback.features)
matching_value(test_feedback, predictions, partial = False, match_rest = False, opt_scipy = False)


(0.9841672461694815, 0.9590556350722312)

In [21]:
import copy

def variation_testing(train_sampler_spec, test_sampler_spec, epochs = 300, model = None, bypass_training = False):
    if model is None and bypass_training:
        print("Need a model to bypass training")
        return


    matching_values = []
    for train_param, test_param in zip(train_sampler_spec, test_sampler_spec):
        # test_param['num_samples'] = 40
        # test_param['batch_size'] = 40
        # schematics = test_param['schematics']
        # schematics[0]['length'] = 1000
        # test_param['schematics'] = schematics

        print("starting test generation")
        test_sampler, _ = samplers(test_param)
        print("finished test generation")

        if not bypass_training:
            train_sampler, spec = samplers(train_param)
            model = define_model(spec, train_sampler, model="mpnn")
            train(model, epochs, train_sampler, test_sampler)
        else:
            print("Bypassing training")

        test_feedback = next(test_sampler)
        predictions, _ = model.predict(rng_key, test_feedback.features)
        accuracy = matching_value(test_feedback, predictions, partial = False, match_rest = False, opt_scipy = True)

        matching_values.append((train_param, test_param, accuracy))
    return model, matching_values

weight_params = [{"low": 0, "high": 0.001},
                 {"low": 1, "high": 1.001},
                 {"low": 1, "high": 1.1},
                 {"low": 1, "high": 2},
                 {"low": 0, "high": 0.1},
                 {"low": 0, "high": 1},
                 # {"low": 0, "high": 10},
                 # {"low": 0, "high": 100},
                 # {"low": 50, "high": 200},
                 # {"low": 500, "high": 2000},
                 # {"low": 5000, "high": 20000}
                 ]


train_sampler_spec = [
    {
        'num_samples': 100, 'batch_size': 32,
        'schematics': [
            {
                'generator': 'ER',
                'proportion': 1,
                'length': 64,
                'kwargs': {'low': 0, 'high': 0.001, 'weighted': True}
            }
        ]
    },
]


test_sampler = [
    {
        'num_samples': 10, 'batch_size': 10,
        'schematics': [
            {
                'generator': 'ER',
                'proportion': 1,
                'length': 1000,
                'kwargs': {'p': 0.1, 'low': 0, 'high': 1, 'weighted': True}
            }
        ]
    },
]


length_training = [{"generator": "ER"}]
length_testing = [{"generator": "ER", "length": 1000, "p": 0.01}]




model, results = variation_testing(train_sampler_spec, copy.deepcopy(test_sampler), model = model, bypass_training = True)

results




starting test generation
finished test generation
Bypassing training


[({'num_samples': 100,
   'batch_size': 32,
   'schematics': [{'generator': 'ER',
     'proportion': 1,
     'length': 64,
     'kwargs': {'low': 0, 'high': 0.001, 'weighted': True}}]},
  {'num_samples': 10,
   'batch_size': 10,
   'schematics': [{'generator': 'ER',
     'proportion': 1,
     'length': 1000,
     'kwargs': {'p': 0.1, 'low': 0, 'high': 1, 'weighted': True}}]},
  (0.8577524175395072, 0.9537646702986899))]

In [9]:
# import copy
# model2 = copy.deepcopy(model)

ER p=0.25, 100 8x8 train and 40 32x32 test => 0.94 in 100 iterations

BA param=3, 100 8x8 train and 40 32x32 test => 0.97 in 100 iterations

BA param=5, 100 8x8 train and 40 32x32 test => 0.95 in 100 iterations (0.951 in 200 so has pretty much converged after 100)

BA param=7, 100 8x8 train and 40 32x32 test => 0.946 in 100 iterations

#### Cross training
BA param=7 to BA param=3

BA param=7 to ER p=0.25 0.946 with BA to 0.939 with ER (same as if trained only on BA)

ER p=0.25 to BA param=3 went from 0.939 with ER to 0.967 with BA (BA param 3 was 0.97 so basically nothing lost)

#### Weight variations
Uniform
* 0,0.001 -> 0.928
* 1,1.001 -> 0.962
* 0,0.1 -> 0.931
* 0,10 -> 0.883
* 0,100 -> 0.77
* 50, 200 -> 0.72
* 500, 2000 -> 0.69
* 5000, 20000 -> 0.7

Normal:
Basically same.

Gumbel
* 0,0.001 -> 0.323
* 1,1.001 -> 0.849
* 0,0.1 -> 0.498
* 5,10 -> 0.82
* 5,100 -> 0.8

#### Weight cross training
Train ER p=0.25 unif 0,1:
* 0,0.001 -> 0.948
* 1,1.001 -> 0.967
* 0,0.1 -> 0.916
* 0,10 -> 0.86
* 0,100 -> 0.75
* 50, 200 -> 0.72
* 500, 2000 -> 0.72
* 5000, 20000 -> 0.69

=> Seems to weight generalize quite well. Actually even better because basically no statistical difference with if we trained separately.

Train normal 5000, 20000:
* 0, 0.001 -> 0.39 (maybe it's the large to small that was a problem here? Also those values make little sense for a normal RV)

Other direction train normal 0, 0.001 (got to 0.78):
* 5000, 20000 -> 0.76

=> small to large seems better


#### Larger graphs
Same training
ER p=0.25 8x8 train:
* 100x100 test goes to 0.88
* 200x200 goes to 0.63 (only 12 prediction mismatches though)
* 200x200 p=0.3 =>
* 250x250 => 0.9448 (BUT p=0.1 to not kill my computer)
*
Try this but 16x16 train

#### RIDESHARE
8x8 train,
* 32x32 test => 0.96
* 50x50 test => 0.96
* 100x100 test => 0.938
* 250x250 test => 0.9

#### Double max
8x8 train 32x23 test
300 iterations gets us to 0.93 as normal max (though normal max takes 100 iterations to get there), 600 iterations gets us to 0.965
==> Testing single max on 600 iterations => 0.956
==> Testing single max with 64 hidden dim embeddings on 600 iterations 0.96 (already in 200) (seeing if gain is only from more parameters or if double max is actually more aligned)

Conclusion, it was mainly due to more iterations + some amount of more parameters but only 1% so probably not statistically significant.

#### Training with scaling
Train/test with 5000, 200000 weights ==> 0.76 accuracy
But if normalize 0, 1 on training (or just train on normalized) ==> 0.91 (same acc as had train/testing on normalized)

#### More weight scales training
300 epochs for all, 4x4 train, 32x32 test
* 0, 1: 0.946, 0.917
* 1, 1.01: 0.993, 0.970
* 1, 1.001: 0.956, 0.976
* 1, 1.1: 0.989, 0.972
* 1, 1.2: 0.986, 0.958
* 1, 1.5: 0.966, 0.949
* 1, 2: 0.957, 0.939
* 2, 2.1: 0.9927, 0.9757
* 10, 10.001: 0.4, 0.97
Realization: shifting just doesn't makes sense (val + 1000) / (opt + 1000) > val / opt

==> find the best range
* 0, 0.001: 0.79, 0.929
* 0, 0.01: 0.79, 0.922
* 0, 0.1: 0.76, 0.934
* 0, 1: 0.94, 0.9334
* 0, 10: 0.897, 0.928
* 0, 100: 0.7, 0.93

### Teacher forcing
1.0:  100 epochs => 0.92   | 200 epochs => 0.968
0.75: 100 epochs => 0.956 | 200 epochs => 0.959 | 300 => 0.957 | 400 => 0.966 | 500 => 0.954 | 600 => 0.964
0.5: 100 epochs => 0.943 | 200 => 0.943 | 300 => 0.952 | 400 => 0.938 | 500 => 0.928 | 0.953
0.25: 100 => 0.923 | 200 => 0.935 | 300 => 0.932 | 400 => 0.936 | 500 => 0.948 | 600 => 0.94
0: 100 => 0.9 | 200 => 0.977 | 300 => 0.937 | 400 => 0.924 | 500 => 0.923 | 600 => 0.958 | 800 => 0.98

### GMPNN
MPNN: 100 => 0.92  | 200 => 0.968
GMPNN 100 => 0.956 | 200 => 0.957 | 300 => 0.948 | 400 => 0.935

### Larger graphs
GPMNN with 16 node (8x8) graphs as training
100 => 0.965 | 200 => 0.968 | 300 => 0.969 | 400 => 0.971 | 500 => 0.972 | 600 => 0.971 | 700 => 0.981 | 800 => 0.973

### Train on larger
GPMNN with 16 node (8x8) graphs as training
100 => 0.965 | 200 => 0.968 | 300 => 0.969 | 400 => 0.971 | 500 => 0.972 | 600 => 0.971 | 700 => 0.981 | 800 => 0.973

### Soft pointers


#### Cross training with p value for ER
GMPNN 400 epochs
ER
0.05: 0.87, 0.96
0.1: 0.76, 0.94
0.2: 0.75, 0.91
0.5: 0.93, 0.93
0.75: 0.94, 0.95
1: 0.88, 0.96

Compare to if learned directly: (train on those parameters then test i.e. no cross-training) + is MPNN, not GMPNN
0.05: 0.95, 0.96
0.1: 0.86, 0.93
0.2: 0.7, 0.9
0.5: 0.9, 0.94
0.75: 0.93, 0.95
1: 0.85, 0.95

Already got ER + Rideshare generalization + BA generalization to other parameters to larger graphs


## Preliminary results
random permutation/matching: 0.18

MPNN:
learned predictions: 0.67

GAT:
learned predictions: 0.72

Got better with double ended predictions

Partial: 0.64 while greedy was doing about 0.92 on the same instance. Main reason seems to be that max weight is around 1.5 => can get at most 2/3 OPT


### Counting the number of matching constraints violated

In [27]:
# For two-way
def count_mismatches_two_way(predictions):
    count = 0
    data = predictions["match"].data
    nb_graphs = data.shape[0]
    for datapoint in range(data.shape[0]):
        for i in range(32):
            owner = data[datapoint][i]
            good = data[datapoint][int(owner)]
            if good != i:
                count += 1
    print(f"average number of edges contradicting matching: {count / nb_graphs}")

average number of edges contradicting matching: 12.2


In [17]:
# For self-loops
def count_mismatches_self_loop(predictions):
    count = 0
    data = predictions["match"].data
    nb_graphs = data.shape[0]
    for datapoint in range(data.shape[0]):
        owners = set(np.array(data[datapoint][32:64]))
        count += 32 - len(owners)
    print(f"average number of edges contradicting matching: {count / nb_graphs}")


average number of edges contradicting matching: 0.4


In [14]:
a = np.array([1, 2])
b = np.array([2, 3])
print(np.concatenate((a, b)))

[1 2 2 3]
